<a href="https://colab.research.google.com/github/Hammadtahir7/Card-Fraud-Detection/blob/master/CCFD_LSTM_SCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving SCD.csv to SCD.csv


In [0]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['SCD.csv']),header=None)

In [3]:
df2.head()

,0,1,2,3,4,5,6,7,8,9
0,100.0,3000.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.0
1,100.0,4300.0,0.0,5.0,1.0,1.0,0.0,0.0,0.0,1.0
2,185.5,4823.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0
3,185.5,5008.5,1.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0
4,500.0,26000.0,0.0,0.0,1.0,1.0,800.0,677.2,6.0,1.0


In [0]:
features =df2.drop(df2.columns[9],axis=1)
labels=df2[df2.columns[9]]

In [5]:
import numpy as np
unique_elements, counts_elements = np.unique(labels, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[0.000e+00 1.000e+00]
 [2.627e+03 4.480e+02]]


In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
features = scaler.fit_transform(features)
print(features[0])

X_train, X_test, y_train, y_test=train_test_split(features,labels,test_size=0.2, random_state=42)

#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[-1.42200799 -0.67856862 -0.13742882  1.84422371  1.83180896  3.74165739
 -0.26978337 -0.25664688 -0.25310881]
(2460, 9)
(615, 9)
(2460,)
(615,)


In [7]:
X_train=X_train.reshape(2460,1,9)
X_test=X_test.reshape(615,1,9)
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1,9)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m,precision_m, recall_m])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)




Train on 2460 samples, validate on 615 samples
Epoch 1/10





2460/2460 [==============================] - 1s 564us/step - loss: 0.5995 - acc: 0.8915 - f1_m: 0.6272 - precision_m: 0.7374 - recall_m: 0.6059 - val_loss: 0.4902 - val_acc: 0.9317 - val_f1_m: 0.7863 - val_precision_m: 0.8370 - val_recall_m: 0.7769
Epoch 2/10
2460/2460 [==============================] - 0s 102us/step - loss: 0.3570 - acc: 0.9537 - f1_m: 0.7957 - precision_m: 0.8157 - recall_m: 0.8110 - val_loss: 0.2474 - val_acc: 0.9512 - val_f1_m: 0.8565 - val_precision_m: 0.8508 - val_recall_m: 0.8958
Epoch 3/10
2460/2460 [==============================] - 0s 113us/step - loss: 0.1768 - acc: 0.9675 - f1_m: 0.8612 - precision_m: 0.8777 - recall_m: 0.8695 - val_loss: 0.1438 - val_acc: 0.9659 - val_f1_m: 0.8916 - val_precision_m: 0.8999 - val_recall_m: 0.9045
Epoch 4/10
2460/2460 [==============================] - 0s 109us/step - loss: 0.1108 - acc: 0.9724 - f1_m: 0.8793 - precision_m: 0.9029 - recall_m: 0.8850 - val_loss

In [11]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


print("loss: "+"{:.2%}".format(loss));
print("acc: "+"{:.2%}".format(accuracy));
print("f1: "+"{:.2%}".format(f1_score));
print("precision: "+"{:.2%}".format(precision));
print("recall: "+"{:.2%}".format(recall));

loss: 6.16%
acc: 98.05%
f1: 93.12%
precision: 94.14%
recall: 93.22%


In [0]:
y_pred=model.predict_classes(X_test)

In [13]:
from sklearn import metrics
print("Accuracy: "+"{:.2%}".format(metrics.accuracy_score(y_test, y_pred)));
print("Precision: "+"{:.2%}".format(metrics.precision_score(y_test, y_pred)));
print("Recall: "+"{:.2%}".format(metrics.recall_score(y_test, y_pred)));
print("F1: "+"{:.2%}".format(metrics.f1_score(y_test,y_pred)));
print(metrics.confusion_matrix(y_test,y_pred));

Accuracy: 98.05%
Precision: 95.05%
Recall: 93.20%
F1: 94.12%
[[507   5]
 [  7  96]]
